In [7]:
from src import Tensor,cross_entropy,Adam,SGD,Linear, Module, Sequential, ReLU,dataloader,Conv2D
import os
import sys
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [8]:
import sklearn.datasets
train_X, train_y = sklearn.datasets.load_digits(return_X_y=True)

In [9]:
class Net(Module):
    def __init__(self, outc, impl):
        super().__init__()
        self.seq = Sequential(
            Conv2D(None, 64, 3, conv_impl=impl),
            ReLU(),
            lambda x: x.reshape(x.shape[0], -1),
            Linear(None, outc,bias=False)
        )

    def forward(self, x):
        x = self.seq(x)
        x = x.reshape(x.shape[0],-1)
        return x


def train(net,optim,train_X,train_y,epochs):
    if train_X.ndim == 2:
        h=w=int(np.sqrt(train_X.shape[1]))
        train_X = train_X.reshape(train_X.shape[0],1,h,w)
    import time
    start = time.monotonic()
    for epoch in range(epochs):  # loop over the dataset multiple times
        losses = []
        accs = []
        for X, y in dataloader(64*2, train_X, train_y):
            # forward + backward + optimize
            outputs = net.forward(X)
            loss = cross_entropy(outputs, y)
            optim.zero_grad()
            loss.backward()
            assert loss.item() >= 0, loss
            optim.step()

            losses.append(loss.item())
            accs.append((outputs.data.argmax(-1) == y.data).astype(float).mean())
        print(
            f'[Epoch {epoch + 1:3}] loss: {np.mean(losses):.3f} accuracy: {np.mean(accs)*100:3.2f}')
    return time.monotonic() - start

In [10]:
net = Net(np.unique(train_y).size, impl="slow")
optimizer = Adam(net.trainable_params(), lr=0.001, amsgrad=True)
time = train(net, optimizer, train_X, train_y, 10)
print(f'Finished Training in : {time:.4f} s')

[Epoch   1] loss: 58.610 accuracy: 45.54
[Epoch   2] loss: 5.874 accuracy: 90.57
[Epoch   3] loss: 2.602 accuracy: 95.10
[Epoch   4] loss: 1.701 accuracy: 95.44
[Epoch   5] loss: 2.207 accuracy: 94.22
[Epoch   6] loss: 1.199 accuracy: 96.82
[Epoch   7] loss: 0.510 accuracy: 98.49
[Epoch   8] loss: 0.312 accuracy: 98.96
[Epoch   9] loss: 0.226 accuracy: 99.22
[Epoch  10] loss: 0.142 accuracy: 99.48
Finished Training in : 14.7970 s


In [11]:
net = Net(np.unique(train_y).size, impl="fast_forward")
optimizer = Adam(net.trainable_params(), lr=0.001, amsgrad=True)
time = train(net, optimizer, train_X, train_y, 10)
print(f'Finished Training in : {time:.4f} s')

[Epoch   1] loss: 50.579 accuracy: 48.61
[Epoch   2] loss: 8.676 accuracy: 86.72
[Epoch   3] loss: 3.505 accuracy: 92.89
[Epoch   4] loss: 2.715 accuracy: 95.52
[Epoch   5] loss: 1.295 accuracy: 97.03
[Epoch   6] loss: 0.767 accuracy: 98.33
[Epoch   7] loss: 0.454 accuracy: 98.59
[Epoch   8] loss: 0.230 accuracy: 98.96
[Epoch   9] loss: 0.148 accuracy: 99.17
[Epoch  10] loss: 0.129 accuracy: 99.01
Finished Training in : 10.7500 s


In [12]:
net = Net(np.unique(train_y).size, impl="fast")
optimizer = Adam(net.trainable_params(), lr=0.001, amsgrad=True)
time = train(net, optimizer, train_X, train_y, 10)
print(f'Finished Training in : {time:.4f} s')

[Epoch   1] loss: 76.765 accuracy: 50.05
[Epoch   2] loss: 18.241 accuracy: 79.24
[Epoch   3] loss: 4.046 accuracy: 94.01
[Epoch   4] loss: 2.149 accuracy: 96.51
[Epoch   5] loss: 1.276 accuracy: 97.81
[Epoch   6] loss: 0.806 accuracy: 98.28
[Epoch   7] loss: 0.544 accuracy: 98.49
[Epoch   8] loss: 0.408 accuracy: 99.11
[Epoch   9] loss: 0.315 accuracy: 98.91
[Epoch  10] loss: 0.262 accuracy: 99.22
Finished Training in : 1.1720 s
